**Downloading Data and Installing Dependencies**

In [ ]:
!gdown --id 18qSvsM5W2QFASUlLWFPqxuYFPZcOs25b
!unzip LUMS_FALL2020_PROJECT_DATA1.zip 
!pip install python_speech_features

**Import and Constants**

In [ ]:
import glob
from functools import partial

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
import python_speech_features as mfcc
from scipy.io.wavfile import read
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from tqdm import tqdm

tqdm = partial(tqdm, position=0, leave=True)

**Helper Functions for Preprocessing**

In [ ]:
def get_MFCC(audio, sr):
    features = mfcc.mfcc(audio, sr, 0.025, 0.01, 13, appendEnergy = True)
    return np.mean(features, axis=0)

def gender_label(filename):
    '''
    Function to extract the label of gender data from filename.
    
    Arguments
    ---------
    filename: str
        Name of the input file
    
    Returns
    -------
    Extracted label of either 'F' or 'M'.
    '''

    return filename.split('/')[-2][-1]

def speaker_label(filename):
    '''
    Function to extract the label of speaker data from filename.
    
    Arguments
    ---------
    filename: str
        Name of the input file
    
    Returns
    -------
    Extracted label in the range of 001-142
    '''

    return filename.split('/')[-2][-5:-2]

def load_data(file_names, key):
    '''
    Function to load data from the files.
    
    Arguments
    ---------
    file_names : list
        A list containing names of all the files from where data and labels will be extracted. 
    key : function
        A function to extract label from the filename.
    
    Returns
    -------
    data : array
        Input data of shape (m,n) where m is the no. of instances and n is the no. of features.
    label : array    
        An array of shape (m, ) containing labels of the input data.
    '''
    
    data, labels = [], []

    for filename in tqdm(file_names):
        sr, audio = read(filename)
        data.append(get_MFCC(audio, sr))
        labels.append(key(filename))

    return np.array(data), np.array(labels)


**Reading filenames of train data and test data using  glob()**


In [ ]:
GR_train_files = glob.glob('Gender_Recognition/Train/*/*.wav')
GR_test_files = glob.glob('Gender_Recognition/Test/*/*.wav')

SP_train_files = glob.glob('Speaker_Recognition/Train/*/*.wav')
SP_test_files = glob.glob('Speaker_Recognition/Test/*/*.wav')

**Loading training data and test data for gender and speaker recognition**

In [ ]:
GR_trainX, GR_trainY = load_data(GR_train_files, gender_label)
GR_testX, GR_testY = load_data(GR_test_files, gender_label)

SP_trainX, SP_trainY = load_data(SP_train_files, speaker_label)
SP_testX, SP_testY = load_data(SP_test_files, speaker_label)

GR_LABELS = sorted(np.unique(GR_trainY))
SP_LABELS = sorted(np.unique(SP_trainY))

100%|██████████| 284/284 [00:10<00:00, 25.94it/s]


# **PART 1 (Multi Layer Perceptron)**

In [ ]:
f1_macro = make_scorer(f1_score , average='macro')
MLP = MLPClassifier(max_iter=6000)
parameters = {
    'random_state': [1],
    'activation': ['logistic'],
    'solver': ['sgd'],
    'learning_rate_init': [0.4, 0.1, 0.01],
    'hidden_layer_sizes': [(128, 64),(64,), (64,32),(32,)],
}
clf = GridSearchCV(MLP, parameters, n_jobs=-1, scoring=f1_macro)

**GENDER RECOGNITION**

In [ ]:
# Finding best learning rate and hidden layer sizes for gender data
GR_clf1 = clf.fit(GR_trainX, GR_trainY)
gr_learning_rate_init, gr_hidden_layer_sizes = GR_clf1.best_params_.get('learning_rate_init'), GR_clf1.best_params_.get('hidden_layer_sizes')
print("Best learning rate init: ",gr_learning_rate_init)
print("Best hidden layer sizes: ",gr_hidden_layer_sizes)

Best learning rate init:  0.01
Best hidden layer sizes:  (128, 64)


In [ ]:
# Using scikit-learn’s Multi-layer Perceptron implementation to train and test the multi-layer neural networks on gender data
GR_NN = MLPClassifier(random_state=1, activation='logistic',solver='sgd', max_iter=5000, learning_rate_init = gr_learning_rate_init, hidden_layer_sizes = gr_hidden_layer_sizes)
GR_NN.fit(GR_trainX, GR_trainY)
GR_predY1 = GR_NN.predict(GR_testX)

In [ ]:
print(f'Accuracy Score: {accuracy_score(GR_testY, GR_predY1):2.4f}')
print("\nClassification Report:")
print(classification_report(GR_testY, GR_predY1, labels=GR_LABELS, ))
print("\nConfusion Matrix:")
gender_confusion = pd.DataFrame(confusion_matrix(GR_testY, GR_predY1), columns=GR_LABELS, index=GR_LABELS)
gender_confusion.columns.name = 'Prediction'
gender_confusion.index.name = 'Target'
gender_confusion

Accuracy Score: 0.8765

Classification Report:
              precision    recall  f1-score   support

           F       0.77      0.68      0.72        40
           M       0.90      0.94      0.92       130

    accuracy                           0.88       170
   macro avg       0.84      0.81      0.82       170
weighted avg       0.87      0.88      0.87       170


Confusion Matrix:


Prediction,F,M
Target,,
F,27,13
M,8,122


**SPEAKER RECOGNITION**

In [ ]:
# Finding best learning rate and hidden layer sizes for speaker data
SP_clf1 = clf.fit(SP_trainX, SP_trainY)
SP_learning_rate_init, SP_hidden_layer_sizes = SP_clf1.best_params_.get('learning_rate_init'), SP_clf1.best_params_.get('hidden_layer_sizes')
print("Best learning rate init: ",SP_learning_rate_init)
print("Best hidden layer sizes: ",SP_hidden_layer_sizes)

Best learning rate init:  0.4
Best hidden layer sizes:  (64,)


In [ ]:
# Using scikit-learn’s Multi-layer Perceptron implementation to train and test the multi-layer neural networks on speaker data
SP_NN = MLPClassifier(max_iter=5000, random_state=1, activation='logistic',solver='sgd', learning_rate_init = SP_learning_rate_init, hidden_layer_sizes = SP_hidden_layer_sizes) 
SP_NN.fit(SP_trainX, SP_trainY)
SP_predY1 = SP_NN.predict(SP_testX)

In [ ]:
print(f'Accuracy score: {accuracy_score(SP_testY, SP_predY1):2.4f}')
print("\nClassification Report:")
print(classification_report(SP_testY, SP_predY1, labels = SP_LABELS, output_dict=True)['macro avg'])
print("\nConfusion Matrix:")
speaker_confusion = pd.DataFrame(confusion_matrix(SP_testY, SP_predY1), columns=SP_LABELS, index=SP_LABELS)
speaker_confusion.columns.name = 'Prediction'
speaker_confusion.index.name = 'Target'
speaker_confusion

Accuracy score: 0.9437

Classification Report:
{'precision': 0.9612676056338029, 'recall': 0.9436619718309859, 'f1-score': 0.9408450704225352, 'support': 284}

Confusion Matrix:


Prediction,001,002,003,004,005,006,007,008,009,010,011,012,013,014,015,016,017,018,019,020,021,022,023,024,025,026,027,028,029,030,031,032,033,034,035,036,037,038,039,040,...,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142
Target,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
003,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
004,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
005,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0


# **PART 2 (Linear Support Vector)**



**GENDER RECOGNITION**

In [ ]:
# Using scikit learn's Linear Support Vector implementation to train and test the linear SVM on gender data
GR_clf2 = LinearSVC(max_iter= 100000)
GR_clf2.fit(GR_trainX, GR_trainY)
GR_predY2 = GR_clf2.predict(GR_testX)

In [ ]:
print(f'Accuracy Score: {accuracy_score(GR_testY, GR_predY2):2.4f}')
print("\nClassification Report:")
print(classification_report(GR_testY, GR_predY2, labels=GR_LABELS))
print("\nConfusion Matrix:")
gender_confusion = pd.DataFrame(confusion_matrix(GR_testY, GR_predY2), columns=GR_LABELS, index=GR_LABELS)
gender_confusion.columns.name = 'Prediction'
gender_confusion.index.name = 'Target'
gender_confusion

Accuracy Score: 0.8353

Classification Report:
              precision    recall  f1-score   support

           F       0.69      0.55      0.61        40
           M       0.87      0.92      0.90       130

    accuracy                           0.84       170
   macro avg       0.78      0.74      0.75       170
weighted avg       0.83      0.84      0.83       170


Confusion Matrix:


Prediction,F,M
Target,,
F,22,18
M,10,120


**SPEAKER RECOGNITION**

In [ ]:
# Using scikit learn's Linear Support Vector implementation to train and test the linear SVM on speaker data
SP_clf2 = LinearSVC(max_iter= 150000)
SP_clf2.fit(SP_trainX, SP_trainY)
SP_predY2 = SP_clf2.predict(SP_testX)

In [ ]:
print(f'Accuracy Score: {accuracy_score(GR_testY, GR_predY2):2.4f}')
print("\nClassification Report:")
print(classification_report(SP_testY, SP_predY2, labels = SP_LABELS, output_dict=True, zero_division=0)['macro avg'])
print("\nConfusion Matrix:")
speaker_confusion = pd.DataFrame(confusion_matrix(SP_testY, SP_predY2), columns=SP_LABELS, index=SP_LABELS)
speaker_confusion.columns.name = 'Prediction'
speaker_confusion.index.name = 'Target'
speaker_confusion

Accuracy Score: 0.8353

Classification Report:
{'precision': 0.8791079812206573, 'recall': 0.8450704225352113, 'f1-score': 0.8368544600938967, 'support': 284}

Confusion Matrix:


Prediction,001,002,003,004,005,006,007,008,009,010,011,012,013,014,015,016,017,018,019,020,021,022,023,024,025,026,027,028,029,030,031,032,033,034,035,036,037,038,039,040,...,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142
Target,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
003,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
005,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0


# **PART 3 (Gaussian Naive Bayes)**

**GENDER RECOGNITION**

In [ ]:
# Using scikit-learn’s Gaussian Naïve Bayes implementation to train and test the Gaussian Naïve Bayes on Gender data
GR_clf3 = GaussianNB()
GR_clf3.fit(GR_trainX, GR_trainY)
GR_predY3 = GR_clf3.predict(GR_testX)

In [ ]:
print(f'Accuracy Score: {accuracy_score(GR_testY, GR_predY3):2.4f}')
print("\nClassification Report:")
print(classification_report(GR_testY, GR_predY3, labels=GR_LABELS ))
print("\nConfusion Matrix:")
gender_confusion = pd.DataFrame(confusion_matrix(GR_testY, GR_predY3), columns=GR_LABELS, index=GR_LABELS)
gender_confusion.columns.name = 'Prediction'
gender_confusion.index.name = 'Target'
gender_confusion

Accuracy Score: 0.8529

Classification Report:
              precision    recall  f1-score   support

           F       0.71      0.62      0.67        40
           M       0.89      0.92      0.91       130

    accuracy                           0.85       170
   macro avg       0.80      0.77      0.79       170
weighted avg       0.85      0.85      0.85       170


Confusion Matrix:


Prediction,F,M
Target,,
F,25,15
M,10,120


**SPEAKER RECOGNITION**

In [ ]:
# Using scikit-learn’s Gaussian Naïve Bayes implementation to train and test the Gaussian Naïve Bayes on Gender data
SP_clf3 = GaussianNB()
SP_clf3.fit(SP_trainX, SP_trainY)
SP_predY3 = SP_clf3.predict(SP_testX)

In [ ]:
print(f'Accuracy Score: {accuracy_score(GR_testY, GR_predY3):2.4f}')
print("\nClassification Report:")
print(classification_report(SP_testY, SP_predY3, output_dict=True, zero_division=0)['macro avg'])
print("\nConfusion Matrix:")
speaker_confusion = pd.DataFrame(confusion_matrix(SP_testY, SP_predY3), columns=SP_LABELS, index=SP_LABELS)
speaker_confusion.columns.name = 'Prediction'
speaker_confusion.index.name = 'Target'
speaker_confusion

Accuracy Score: 0.8529

Classification Report:
{'precision': 0.9417840375586856, 'recall': 0.9225352112676056, 'f1-score': 0.9174044265593562, 'support': 284}

Confusion Matrix:


Prediction,001,002,003,004,005,006,007,008,009,010,011,012,013,014,015,016,017,018,019,020,021,022,023,024,025,026,027,028,029,030,031,032,033,034,035,036,037,038,039,040,...,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142
Target,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
002,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
003,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
004,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
005,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0
139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
